****Ingest Website to Graph DB****

**Part 3** - Extract Entities & Relationships using Langchain

Extract Entities and Relatonships from a body of Text using langchain.

This is a GCP reworking of

https://python.langchain.com/docs/use_cases/graph/constructing/#llm-graph-transformer

**Minimal install for Vertex AI**

This solved the instability problem by *NOT* installing OpenAI classes via the community install. 

In [1]:
pip install -U langchain-google-vertexai langchain-experimental neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


**Check Version Nos of what was installed**

In [2]:
!pip show langchain-core langchain-google-vertexai langchain-experimental neo4j

Name: langchain-core
Version: 0.1.52
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity
Required-by: langchain, langchain-community, langchain-experimental, langchain-google-genai, langchain-google-vertexai, langchain-text-splitters
---
Name: langchain-google-vertexai
Version: 1.0.3
Summary: An integration package connecting Google VertexAI and LangChain
Home-page: https://github.com/langchain-ai/langchain-google
Author: 
Author-email: 
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: google-cloud-aiplatform, google-cloud-storage, langchain-core, types-protobuf, types-requests
Required-by: 
---
Name: langchain-experimental
Version: 0.0.58
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/l

**Check Jupyter Version No**

In [3]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.21.0
ipykernel        : 6.29.4
ipywidgets       : 8.1.2
jupyter_client   : 7.4.9
jupyter_core     : 5.7.2
jupyter_server   : 1.24.0
jupyterlab       : 3.4.8
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : 6.5.7
qtconsole        : not installed
traitlets        : 5.14.3


**Check Python Version/Path** - *Expect 3.10.14*

In [4]:
import sys
import platform
print(sys.version)
print(platform.python_version())
print(sys.path)

3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
3.10.14
['/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages']


**Now for the Imports**

This time we are isloating Vertex AI

In [5]:
import os
import requests

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_vertexai import ChatVertexAI
from langchain_core.documents import Document

**Diagnostic Methods**

In [6]:
## Vertex AI Model Graph Transformer Diagnostic Dump
def print_llm_xfrm(llm_xfrm):
    print(f"llm_xfrm: {llm_xfrm}") 
    print(f"allowed_nodes: {llm_xfrm.allowed_nodes}") 
    print(f"allowed_relationships: {llm_xfrm.allowed_relationships}") 
    print(f"strict_mode: {llm_xfrm.strict_mode}")     

In [7]:
## Vertex AI Model Diagnostic Dump
def print_llm(llm):
    print(f"llm: {llm}")
    print(f"name: {llm.model_name}")
    print(f"examples: {llm.examples}") 
    print(f"tuned_model_name: {llm.tuned_model_name}") 
    print(f"convert_system_message_to_human: {llm.convert_system_message_to_human}") 
    print(f"max_output_tokens: {llm.max_output_tokens}") 
    print(f"top_p: {llm.top_p}") 
    print(f"top_k: {llm.top_k}") 
    print(f"credentials: {llm.credentials}")     
    print(f"n: {llm.n}") 
    print(f"streaming: {llm.streaming}") 
    print(f"safety_settings: {llm.safety_settings}")     
    print(f"api_transport: {llm.api_transport}") 
    print(f"api_endpoint: {llm.api_endpoint}") 
 
    print('properties') 
    print('----------') 
    
    print(f"_llm_type: {llm._llm_type}") 
    print(f"is_codey_model: {llm.is_codey_model}")     
    print(f"_is_gemini_model: {llm._is_gemini_model}")
    print(f"_identifying_params: {llm._identifying_params}")     
    print(f"_default_params: {llm._default_params}") 
    print(f"_user_agent: {llm._user_agent}") 



In [8]:
def print_graph_documents(graph_documents):
    print(f"graph_documents_len: {len(graph_documents)}") 
    graphdoc_idx = 0
    for gdoc in graph_documents:
        print(" ") 
        print(f"graphdoc_idx: {graphdoc_idx}") 
        graphdoc_idx += 1
        
        print(f"Len doc_page_content {len(gdoc.source.page_content)}") 
        print(f"No. doc_metadata: {len(gdoc.source.metadata)}")        
        
        print(f"No. nodes: {len(gdoc.nodes)}") 
        for noddy in gdoc.nodes:
            print(f"Node: {noddy}")
        
        print(f"No. relationships: {len(gdoc.relationships)}") 
        for relly in gdoc.relationships:
            print(f"Relationship: {relly}")
        


**Connect to Google LLMs**

*Least Privilege Security.*

The Notebook is "owned" by a bespoke Service Account created in terrafrom for this purpose.

Minimal permisisons are added (also via terraform) via predefined roles (esp. Vertex) as required.

This is typically triggered by a PERMISSION DENIED error

In [9]:
import os
# Set It - will require regeneration
os.environ['GOOGLE_API_KEY'] = 'xxx'
# Access the environment variable later in your code
api_key = os.environ['GOOGLE_API_KEY']
print(api_key)

1c6f2b6627a6125776a0a76a644ccc040172495a


**Create The LLMs**

Both *Gemini* & *Chat Bison* were created.

Chat Bison malfunctioned so has been abandoned FTTB 

Sourced from here: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models 

In [10]:
# Avalable Model Variables

# Gemini 1.5 Pro (Preview)
# 404 Publisher Model `projects/nlp-dev-6aae/locations/us-central1/publishers/google/models/gemini-1.5-pro` not found.
#gemini_1pt5_proOnVertex = ChatVertexAI(model="gemini-1.5-pro")
#Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 10.0 seconds as it raised InvalidArgument: 400 Request contains an invalid argument..
gemini_1pt5_proOnVertex = ChatVertexAI(model="gemini-1.5-pro-preview-0409")



# Gemini 1.0 Pro
# This works with Errors - chunking
gemini_1pt0_proOnVertex = ChatVertexAI(model="gemini-1.0-pro")

gemini_proOnVertex = ChatVertexAI(model="gemini-pro")

# PaLM 2 for Chat ("chat-bison")
# This fails atm
model_chat_bison = ChatVertexAI()


**Choose which model we are using**

In [11]:
chat_llm = gemini_1pt0_proOnVertex

*Check Model Properties*

In [12]:
print_llm(chat_llm)

llm: client=<vertexai.generative_models.GenerativeModel object at 0x7f2b9881c310> model_name='gemini-1.0-pro' client_preview=<vertexai.generative_models.GenerativeModel object at 0x7f2b9881c7c0>
name: gemini-1.0-pro
examples: None
tuned_model_name: None
convert_system_message_to_human: False
max_output_tokens: None
top_p: None
top_k: None
credentials: None
n: 1
streaming: False
safety_settings: None
api_transport: None
api_endpoint: None
properties
----------
_llm_type: vertexai
is_codey_model: False
_is_gemini_model: True
_identifying_params: {'model_name': 'gemini-1.0-pro', 'candidate_count': 1}
_default_params: {'candidate_count': 1}
_user_agent: langchain-google-vertexai/1.0.3-ChatVertexAI_gemini-1.0-pro


**Construct Graph Transformer**

In [13]:
llm_transformer = LLMGraphTransformer(llm=chat_llm)

In [14]:
print_llm_xfrm(llm_transformer)

llm_xfrm: <langchain_experimental.graph_transformers.llm.LLMGraphTransformer object at 0x7f2ba12ede40>
allowed_nodes: []
allowed_relationships: []
strict_mode: True


**Easy Test Case**

In [15]:
from langchain_core.documents import Document

text = """
Current Professional Machine Learning Engineer Certification exam guide
A Professional Machine Learning Engineer builds, evaluates, productionizes, and optimizes ML models by using Google Cloud technologies and knowledge of proven models and techniques. The ML Engineer handles large, complex datasets and creates repeatable, reusable code. The ML Engineer considers responsible AI and fairness throughout the ML model development process, and collaborates closely with other job roles to ensure long-term success of ML-based applications. The ML Engineer has strong programming skills and experience with data platforms and distributed data processing tools. The ML Engineer is proficient in the areas of model architecture, data and ML pipeline creation, and metrics interpretation. The ML Engineer is familiar with foundational concepts of MLOps, application development, infrastructure management, data engineering, and data governance. The ML Engineer makes ML accessible and enables teams across the organization. By training, retraining, deploying, scheduling, monitoring, and improving models, the ML Engineer designs and creates scalable, performant solutions.
Note: The exam does not directly assess coding skill. If you have a minimum proficiency in Python and Cloud SQL, you should be able to interpret any questions with code snippets.
Register now
The Professional Machine Learning Engineer exam does not cover generative AI, as the tools used to develop generative AI-based solutions are evolving quickly. If you are interested in generative AI, please refer to the Introduction to Generative AI Learning Path (all audiences) or the Generative AI for Developers Learning Path (technical audience). If you are a partner, please refer to the Gen AI partner courses: Introduction to Generative AI Learning Path, Generative AI for ML Engineers, and Generative AI for Developers.
Section 1: Architecting low-code ML solutions (~12% of the exam)
1.1 Developing ML models by using BigQuery ML. Considerations include:
Building the appropriate BigQuery ML model (e.g., linear and binary classification, regression, time-series, matrix factorization, boosted trees, autoencoders) based on the business problem
Feature engineering or selection by using BigQuery ML
Generating predictions by using BigQuery ML
1.2 Building AI solutions by using ML APIs. Considerations include:
Building applications by using ML APIs (e.g., Cloud Vision API, Natural Language API, Cloud Speech API, Translation)
Building applications by using industry-specific APIs (e.g., Document AI API, Retail API)
1.3 Training models by using AutoML. Considerations include:
Preparing data for AutoML (e.g., feature selection, data labeling, Tabular Workflows on AutoML)
Using available data (e.g., tabular, text, speech, images, videos) to train custom models
Using AutoML for tabular data
Creating forecasting models using AutoML
Configuring and debugging trained models
Section 2: Collaborating within and across teams to manage data and models (~16% of the exam)
2.1 Exploring and preprocessing organization-wide data (e.g., Cloud Storage, BigQuery, Spanner, Cloud SQL, Apache Spark, Apache Hadoop). Considerations include:
Organizing different types of data (e.g., tabular, text, speech, images, videos) for efficient training
Managing datasets in Vertex AI
Data preprocessing (e.g., Dataflow, TensorFlow Extended [TFX], BigQuery)
Creating and consolidating features in Vertex AI Feature Store
Privacy implications of data usage and/or collection (e.g., handling sensitive data such as personally identifiable information [PII] and protected health information [PHI])
2.2 Model prototyping using Jupyter notebooks. Considerations include:
Choosing the appropriate Jupyter backend on Google Cloud (e.g., Vertex AI Workbench, notebooks on Dataproc)
Applying security best practices in Vertex AI Workbench
Using Spark kernels
Integration with code source repositories
Developing models in Vertex AI Workbench by using common frameworks (e.g., TensorFlow, PyTorch, sklearn, Spark, JAX)
2.3 Tracking and running ML experiments. Considerations include:
Choosing the appropriate Google Cloud environment for development and experimentation (e.g., Vertex AI Experiments, Kubeflow Pipelines, Vertex AI TensorBoard with TensorFlow and PyTorch) given the framework
Section 3: Scaling prototypes into ML models (~18% of the exam)
3.1 Building models. Considerations include:
Choosing ML framework and model architecture
Modeling techniques given interpretability requirements
3.2 Training models. Considerations include:
Organizing training data (e.g., tabular, text, speech, images, videos) on Google Cloud (e.g., Cloud Storage, BigQuery)
Ingestion of various file types (e.g., CSV, JSON, images, Hadoop, databases) into training
Training using different SDKs (e.g., Vertex AI custom training, Kubeflow on Google Kubernetes Engine, AutoML, tabular workflows)
Using distributed training to organize reliable pipelines
Hyperparameter tuning
Troubleshooting ML model training failures
3.3 Choosing appropriate hardware for training. Considerations include:
Evaluation of compute and accelerator options (e.g., CPU, GPU, TPU, edge devices)
Distributed training with TPUs and GPUs (e.g., Reduction Server on Vertex AI, Horovod)
Section 4: Serving and scaling models (~19% of the exam)
4.1 Serving models. Considerations include:
Batch and online inference (e.g., Vertex AI, Dataflow, BigQuery ML, Dataproc)
Using different frameworks (e.g., PyTorch, XGBoost) to serve models
Organizing a model registry
A/B testing different versions of a model
4.2 Scaling online model serving. Considerations include:
Vertex AI Feature Store
Vertex AI public and private endpoints
Choosing appropriate hardware (e.g., CPU, GPU, TPU, edge)
Scaling the serving backend based on the throughput (e.g., Vertex AI Prediction, containerized serving)
Tuning ML models for training and serving in production (e.g., simplification techniques, optimizing the ML solution for increased performance, latency, memory, throughput)
Section 5: Automating and orchestrating ML pipelines (~21% of the exam)
5.1 Developing end-to-end ML pipelines. Considerations include:
Data and model validation
Ensuring consistent data pre-processing between training and serving
Hosting third-party pipelines on Google Cloud (e.g., MLFlow)
Identifying components, parameters, triggers, and compute needs (e.g., Cloud Build, Cloud Run)
Orchestration framework (e.g., Kubeflow Pipelines, Vertex AI Pipelines, Cloud Composer)
Hybrid or multicloud strategies
System design with TFX components or Kubeflow DSL (e.g., Dataflow)
5.2 Automating model retraining. Considerations include:
Determining an appropriate retraining policy
Continuous integration and continuous delivery (CI/CD) model deployment (e.g., Cloud Build, Jenkins)
5.3 Tracking and auditing metadata. Considerations include: 
Tracking and comparing model artifacts and versions (e.g., Vertex AI Experiments, Vertex ML Metadata)
Hooking into model and dataset versioning
Model and data lineage
Section 6: Monitoring ML solutions (~14% of the exam)
6.1 Identifying risks to ML solutions. Considerations include:
Building secure ML systems (e.g., protecting against unintentional exploitation of data or models, hacking)
Aligning with Google’s Responsible AI practices (e.g., biases)
Assessing ML solution readiness (e.g., data bias, fairness)
Model explainability on Vertex AI (e.g., Vertex AI Prediction)
6.2 Monitoring, testing, and troubleshooting ML solutions. Considerations include:
Establishing continuous evaluation metrics (e.g., Vertex AI Model Monitoring, Explainable AI)
Monitoring for training-serving skew
Monitoring for feature attribution drift
Monitoring model performance against baselines, simpler models, and across the time dimension
Common training and serving errors
"""
documents = [Document(page_content=text)]

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# split documents into text and embeddings

text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=512, 
   chunk_overlap=20,
   length_function=len,
   is_separator_regex=False
)

chunks = text_splitter.split_documents(documents)

print(f"No. chunks{len(chunks)}")

No. chunks20


**Use Gemini** 

Works but with intermittent error which may cause data loss.

Hence need for chunking per variouis discisions forums &c.  

In [17]:
#graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph_documents = llm_transformer.convert_to_graph_documents(chunks)

Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InternalServerError: 500 Internal error occurred..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InternalServerError: 500 Internal error occurred..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InternalServerError: 500 Internal error occurred..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InternalServerError: 500 Internal error occurred..
Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InternalServerError: 500 Internal error occurred..
Retrying langchain_google_vertexai.chat_models._completion_with_r

Check content of **Graph Documents**

In [18]:
print_graph_documents(graph_documents)

graph_documents_len: 20
 
graphdoc_idx: 0
Len doc_page_content 71
No. doc_metadata: 0
No. nodes: 1
Node: id='Machine Learning Engineer Certification' type='Exam'
No. relationships: 0
 
graphdoc_idx: 1
Len doc_page_content 506
No. doc_metadata: 0
No. nodes: 0
No. relationships: 0
 
graphdoc_idx: 2
Len doc_page_content 507
No. doc_metadata: 0
No. nodes: 0
No. relationships: 0
 
graphdoc_idx: 3
Len doc_page_content 113
No. doc_metadata: 0
No. nodes: 0
No. relationships: 0
 
graphdoc_idx: 4
Len doc_page_content 191
No. doc_metadata: 0
No. nodes: 0
No. relationships: 0
 
graphdoc_idx: 5
Len doc_page_content 511
No. doc_metadata: 0
No. nodes: 9
Node: id='Generative Ai' type='Concept'
Node: id='Professional Machine Learning Engineer Exam' type='Concept'
Node: id='Generative Ai Tools' type='Concept'
Node: id='Generative Ai Courses' type='Concept'
Node: id='Generative Ai Learning Path' type='Concept'
Node: id='Introduction To Generative Ai Learning Path' type='Concept'
Node: id='Generative Ai F

**Enter Node4J**

Node4J Connectivity

Requires singing up for free version.

DB Will be stopped if not recently used and will require resuming else will fail. 

In [ ]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "neo4j+s://a657168d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "VM3A9Mz6usNT99nLs_lqQssfVK8JxeD81DnEiXlDkZU"

graph = Neo4jGraph()

**Add to GraphDB**

This statement loads Nodes & Relatonships into Node4J

Thence they can be viewed/manipulated directly on the DB. 

In [ ]:
graph.add_graph_documents(graph_documents)